In [ ]:
import transformers
from transformers import AutoTokenizer, LlamaForCausalLM, AutoModelForCausalLM

model_name_or_path = "/home/jye/huggingface/pretrained_model/Meta-Llama-3-8B-Instruct/"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path, )

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [ ]:
import transformers
import torch
import os
from modelscope import snapshot_download
model_id = snapshot_download("LLM-Research/Meta-Llama-3-8B-Instruct")
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.float16},
    device="cuda:6",
)

In [ ]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
)

terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

outputs = pipeline(
        prompt,
        max_new_tokens=2048,
        eos_token_id=terminators,
        do_sample=False,
        temperature=0.6,
        top_p=1,
        repetition_penalty=1.05
)
print(outputs[0]["generated_text"][len(prompt):])


In [ ]:
from vllm import LLM, SamplingParams
import torch

# model_id = "/home/jye/huggingface/pretrained_model/Meta-Llama-3-8B-Instruct"

llm = LLM(
    model=model_id,
    tensor_parallel_size=2,
    dtype=torch.float16,
)

tokenizer = llm.get_tokenizer()

conversations = tokenizer.apply_chat_template(
    [{'role':'user', 'content':'Hello!'}],
    tokenize=False,
)

outputs = llm.generate(
    [conversations],
    SamplingParams(
        max_length=1048,
        temperature=0.6,
        top_p=0.9,
        repetition_penalty=1.05,
        stop_token_ids=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>"), 128001, 128009]
    )
)